How to add Sequence generated surrogate key as a column          

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession \
    .builder \
    .getOrCreate()

sc = spark.sparkContext

Read File

In [20]:
df = spark.read.csv('/home/phillipefs/spark_dev/pyspark-real-time-scenarios/outfiles/emp/*/*/', header=True, nullValue='null')
df = df.na.drop().dropDuplicates(['EMPNO'])
df.show()

+-----+------+--------+----+----------+----+----+------+------------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+------+--------+----+----------+----+----+------+------------+
| 1234|SEKHAR|  doctor|7777|9999-12-31| 667|  78|    80|  2022-01-03|
| 7499| ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-01|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-01|
| 7654|MARTIN|SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|
| 7844|TURNER|SALESMAN|7698|1981-08-09|1500|   0|    30|  2022-01-02|
+-----+------+--------+----+----------+----+----+------+------------+



monotonically_increasing_id()

In [25]:
df_emp = df.withColumn("id", monotonically_increasing_id()+1)
df_emp.show()

+-----+------+--------+----+----------+----+----+------+------------+---+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE| id|
+-----+------+--------+----+----------+----+----+------+------------+---+
| 1234|SEKHAR|  doctor|7777|9999-12-31| 667|  78|    80|  2022-01-03|  1|
| 7499| ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-01|  2|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-01|  3|
| 7654|MARTIN|SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|  4|
| 7844|TURNER|SALESMAN|7698|1981-08-09|1500|   0|    30|  2022-01-02|  5|
+-----+------+--------+----+----------+----+----+------+------------+---+



row_number()

In [31]:
df_emp = df.withColumn('row_number', row_number().over(Window.partitionBy(lit('')).orderBy(lit(''))))
df_emp.show()

+-----+------+--------+----+----------+----+----+------+------------+----------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|row_number|
+-----+------+--------+----+----------+----+----+------+------------+----------+
| 1234|SEKHAR|  doctor|7777|9999-12-31| 667|  78|    80|  2022-01-03|         1|
| 7499| ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-01|         2|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-01|         3|
| 7654|MARTIN|SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|         4|
| 7844|TURNER|SALESMAN|7698|1981-08-09|1500|   0|    30|  2022-01-02|         5|
+-----+------+--------+----+----------+----+----+------+------------+----------+



crc32

In [32]:
df_emp = df.withColumn("crc_key", crc32('EMPNO'))
df_emp.show()

+-----+------+--------+----+----------+----+----+------+------------+----------+
|EMPNO| ENAME|     JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|   crc_key|
+-----+------+--------+----+----------+----+----+------+------------+----------+
| 1234|SEKHAR|  doctor|7777|9999-12-31| 667|  78|    80|  2022-01-03|2615402659|
| 7499| ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-01|1046173690|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-01|3535170612|
| 7654|MARTIN|SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|4024152101|
| 7844|TURNER|SALESMAN|7698|1981-08-09|1500|   0|    30|  2022-01-02|4234062958|
+-----+------+--------+----+----------+----+----+------+------------+----------+



md5

In [57]:
df_emp = df.withColumn("md5_key", md5('EMPNO').cast('string'))
df_emp.show(truncate=False)

+-----+------+--------+----+----------+----+----+------+------------+--------------------------------+
|EMPNO|ENAME |JOB     |MGR |HIREDATE  |SAL |COMM|DEPTNO|UPDATED_DATE|md5_key                         |
+-----+------+--------+----+----------+----+----+------+------------+--------------------------------+
|1234 |SEKHAR|doctor  |7777|9999-12-31|667 |78  |80    |2022-01-03  |81dc9bdb52d04dc20036dbd8313ed055|
|7499 |ALLEN |SALESMAN|7698|1981-02-20|1600|300 |30    |2022-01-01  |7a2b33c672ce223b2aa5789171ddde2f|
|7521 |WARD  |SALESMAN|7698|1981-02-22|1250|500 |30    |2022-01-01  |e1e1f667ce4596e5644be6fab627c226|
|7654 |MARTIN|SALESMAN|7698|1981-09-21|1250|1400|30    |2022-01-03  |e2a7555f7cabd6e31aef45cb8cda4999|
|7844 |TURNER|SALESMAN|7698|1981-08-09|1500|0   |30    |2022-01-02  |b356e7aed7ee82589e54a466e0dca157|
+-----+------+--------+----+----------+----+----+------+------------+--------------------------------+



sha2()

In [58]:
df_emp = df.withColumn("sha2_key", sha2(col('EMPNO').cast('string'), 256))
df_emp.show(truncate=False)

+-----+------+--------+----+----------+----+----+------+------------+----------------------------------------------------------------+
|EMPNO|ENAME |JOB     |MGR |HIREDATE  |SAL |COMM|DEPTNO|UPDATED_DATE|sha2_key                                                        |
+-----+------+--------+----+----------+----+----+------+------------+----------------------------------------------------------------+
|1234 |SEKHAR|doctor  |7777|9999-12-31|667 |78  |80    |2022-01-03  |03ac674216f3e15c761ee1a5e255f067953623c8b388b4459e13f978d7c846f4|
|7499 |ALLEN |SALESMAN|7698|1981-02-20|1600|300 |30    |2022-01-01  |4427dc2e32a1d099dbe2e3c093a8726e0ea72b9422c36a99915dae1d31e9385f|
|7521 |WARD  |SALESMAN|7698|1981-02-22|1250|500 |30    |2022-01-01  |74ed8ca63e8b4fb8b8ac06e8df400f098b0b09cf1b89c8a331e72e1919b57bd4|
|7654 |MARTIN|SALESMAN|7698|1981-09-21|1250|1400|30    |2022-01-03  |b969b01c158ebfecd0ac568aee526cb1ff8811fcdc77b4d682dab89146ad9891|
|7844 |TURNER|SALESMAN|7698|1981-08-09|1500|0   |30    